In [34]:
import sys

sys.path.append("..")
from piece import piece

piece.start(should_send_to_score=False)

In [35]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.synth_player import SynthNote
from soundmining_tools.generative import *
import random
from enum import StrEnum, Enum
import math
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipycanvas import Canvas, hold_canvas

SOUND_PATH = (
    "/Users/danielstahl/Documents/Music/Pieces/Concrete Music/Concrete Music 11/sounds/Concrete Music 11_sounds"
)
IR_PATH = "/Users/danielstahl/Documents/Music/impulse-response/convolution-ir"

SoundType = StrEnum("SoundType", ["LOW", "MIDDLE", "HIGH"])

Sound = StrEnum(
    "Sound",
    [
        "LONG_RATTLE_1",
        "LONG_RATTLE_2",
        "LONG_SCRATCH_1",
        "LONG_SCRATCH_2",
        "MIDDLE_SCRATCH_1",
        "POT_HIT_LONG_1",
        "POT_HIT_LONG_2",
        "POT_HIT_SHORT_1",
        "POT_HIT_SHORT_2",
        "POT_HIT_SHORT_3",
        "POT_HIT_SHORT_4",
        "POT_HIT_SHORT_5",
        "POT_HIT_SHORT_6",
        "POT_HIT_SHORT_7",
        "POT_HIT_SHORT_8",
        "POT_HIT_SHORT_FLAM_1",
        "POT_HIT_SHORT_FLAM_2",
        "POT_HIT_SHORT_FLAM_3",
        "SCRATCH_HIT_1",
        "SHORT_RATTLE_VARIANT_1_1",
        "SHORT_RATTLE_VARIANT_1_2",
        "SHORT_RATTLE_VARIANT_1_3",
        "SHORT_RATTLE_VARIANT_2_1",
        "SHORT_RATTLE_VARIANT_2_2",
        "SHORT_RATTLE_VARIANT_2_3",
        "SHORT_REPEATED_RATTLES_1",
        "SHORT_SCRATCH_1",
    ],
)

SoundGroup = StrEnum(
    "SoundGroup",
    [
        "LONG_RATTLE",
        "LONG_SCRATCH",
        "MIDDLE_SCRATCH",
        "POT_HIT_LONG",
        "POT_HIT_SHORT",
        "POT_HIT_SHORT_FLAM",
        "SCRATCH_HIT",
        "SHORT_RATTLE_VARIANT_1",
        "SHORT_RATTLE_VARIANT_2",
        "SHORT_REPEATED_RATTLES",
        "SHORT_SCRATCH",
    ],
)

sound_groups = {
    SoundGroup.LONG_RATTLE: [Sound.LONG_RATTLE_1, Sound.LONG_RATTLE_2],
    SoundGroup.LONG_SCRATCH: [Sound.LONG_SCRATCH_1, Sound.LONG_SCRATCH_2],
    SoundGroup.MIDDLE_SCRATCH: [Sound.MIDDLE_SCRATCH_1],
    SoundGroup.POT_HIT_LONG: [Sound.POT_HIT_LONG_1, Sound.POT_HIT_LONG_2],
    SoundGroup.POT_HIT_SHORT: [
        Sound.POT_HIT_SHORT_1,
        Sound.POT_HIT_SHORT_2,
        Sound.POT_HIT_SHORT_3,
        Sound.POT_HIT_SHORT_4,
        Sound.POT_HIT_SHORT_5,
        Sound.POT_HIT_SHORT_6,
        Sound.POT_HIT_SHORT_7,
        Sound.POT_HIT_SHORT_8,
    ],
    SoundGroup.POT_HIT_SHORT_FLAM: [Sound.POT_HIT_SHORT_FLAM_1, Sound.POT_HIT_SHORT_FLAM_2, Sound.POT_HIT_SHORT_FLAM_3],
    SoundGroup.SCRATCH_HIT: [Sound.SCRATCH_HIT_1],
    SoundGroup.SHORT_RATTLE_VARIANT_1: [
        Sound.SHORT_RATTLE_VARIANT_1_1,
        Sound.SHORT_RATTLE_VARIANT_1_2,
        Sound.SHORT_RATTLE_VARIANT_1_3,
    ],
    SoundGroup.SHORT_RATTLE_VARIANT_2: [
        Sound.SHORT_RATTLE_VARIANT_2_1,
        Sound.SHORT_RATTLE_VARIANT_2_2,
        Sound.SHORT_RATTLE_VARIANT_2_3,
    ],
    SoundGroup.SHORT_REPEATED_RATTLES: [Sound.SHORT_REPEATED_RATTLES_1],
    SoundGroup.SHORT_SCRATCH: [Sound.SHORT_SCRATCH_1],
}

sounds = {
    Sound.LONG_RATTLE_1: {},
    Sound.LONG_RATTLE_2: {},
    Sound.LONG_SCRATCH_1: {
        SoundType.LOW: [95, 147, 190, 242, 349, 470],
        SoundType.MIDDLE: [650, 708, 727, 844, 893],
        SoundType.HIGH: [
            1002,
            1058,
            1196,
            1308,
            1406,
            1499,
            1640,
            1660,
            1893,
            3111,
            3536,
            3833,
            5911,
            6423,
            7044,
            7343,
            8738,
            10993,
        ],
    },
    Sound.LONG_SCRATCH_2: {
        SoundType.LOW: [56, 102, 145, 163, 214, 231, 361, 425, 491],
        SoundType.MIDDLE: [534, 607, 632, 680, 750, 783, 848],
        SoundType.HIGH: [
            964,
            1081,
            1170,
            1290,
            1430,
            1599,
            1733,
            3096,
            3394,
            3583,
            6026,
            6370,
            6937,
            7289,
            8726,
            10123,
            11018,
        ],
    },
    Sound.MIDDLE_SCRATCH_1: {},
    Sound.POT_HIT_LONG_1: {
        SoundType.LOW: [181, 325],
        SoundType.MIDDLE: [608, 889],
        SoundType.HIGH: [1148, 1360, 1643, 1992, 2507, 2980, 3251, 3679, 4147, 5907, 7173, 9115, 11043],
    },
    Sound.POT_HIT_LONG_2: {
        SoundType.LOW: [62, 160, 187, 327, 399],
        SoundType.MIDDLE: [545, 606, 795],
        SoundType.HIGH: [
            932,
            1098,
            1320,
            1424,
            1637,
            2136,
            2030,
            2978,
            3139,
            4146,
            5079,
            5809,
            6936,
            8645,
            9988,
            11482,
        ],
    },
    Sound.POT_HIT_SHORT_1: {
        SoundType.LOW: [187],
        SoundType.MIDDLE: [564, 768, 866],
        SoundType.HIGH: [936, 1009, 1195, 1290, 1639, 2927, 3115],
    },
    Sound.POT_HIT_SHORT_2: {
        SoundType.LOW: [353],
        SoundType.MIDDLE: [634, 752, 870],
        SoundType.HIGH: [960, 1033, 1120, 1501, 2086, 2461, 3774],
    },
    Sound.POT_HIT_SHORT_3: {
        SoundType.LOW: [140, 352, 492],
        SoundType.MIDDLE: [657, 770, 869],
        SoundType.HIGH: [960, 1032, 1127, 1192, 1312, 1758, 2273, 2485, 3939, 5015],
    },
    Sound.POT_HIT_SHORT_4: {
        SoundType.LOW: [95, 117, 163, 257],
        SoundType.MIDDLE: [421, 491, 750, 843],
        SoundType.HIGH: [913, 1055, 1219, 1617, 2180, 2508, 3140, 3819, 5414],
    },
    Sound.POT_HIT_SHORT_5: {
        SoundType.LOW: [140, 352, 492],
        SoundType.MIDDLE: [770, 867],
        SoundType.HIGH: [960, 1032, 1192, 1265, 1312, 1758, 2273, 2485, 3029, 3820],
    },
    Sound.POT_HIT_SHORT_6: {
        SoundType.LOW: [92, 140, 213, 328],
        SoundType.MIDDLE: [702, 822],
        SoundType.HIGH: [913, 1054, 1265, 1523, 2297, 3421, 3796, 5976],
    },
    Sound.POT_HIT_SHORT_7: {
        SoundType.LOW: [117, 212, 305, 446],
        SoundType.MIDDLE: [588, 702, 821],
        SoundType.HIGH: [1126, 1622, 2019, 2671, 3797, 4008, 5814, 6422, 10406],
    },
    Sound.POT_HIT_SHORT_8: {
        SoundType.LOW: [74, 132, 218, 347],
        SoundType.MIDDLE: [606, 685, 749, 821],
        SoundType.HIGH: [953, 1076, 1241, 1733, 2884, 3775, 5578, 6304, 9911],
    },
    Sound.POT_HIT_SHORT_FLAM_1: {},
    Sound.POT_HIT_SHORT_FLAM_2: {},
    Sound.POT_HIT_SHORT_FLAM_3: {},
    Sound.SCRATCH_HIT_1: {},
    Sound.SHORT_RATTLE_VARIANT_1_1: {
        SoundType.LOW: [256, 374, 427],
        SoundType.MIDDLE: [688, 843],
        SoundType.HIGH: [960, 1150, 2162, 2285, 2770, 3587, 5070, 6447, 7716, 8594, 9394, 10785, 11416, 13795],
    },
    Sound.SHORT_RATTLE_VARIANT_1_2: {
        SoundType.LOW: [379, 451, 506],
        SoundType.MIDDLE: [703],
        SoundType.HIGH: [1080, 1757, 2595, 3769, 4806, 5113, 5790, 6610, 7172, 8579, 9798, 10991, 11483, 12774],
    },
    Sound.SHORT_RATTLE_VARIANT_1_3: {
        SoundType.LOW: [312, 365, 375, 492],
        SoundType.MIDDLE: [667, 838],
        SoundType.HIGH: [1062, 1757, 2629, 5029, 6447, 7217, 8510, 9422, 10473, 11459, 13594],
    },
    Sound.SHORT_RATTLE_VARIANT_2_1: {
        SoundType.LOW: [143, 282, 328, 395, 559],
        SoundType.MIDDLE: [752],
        SoundType.HIGH: [1106, 1661, 2279, 2959, 3851, 5438, 6073, 6376, 7079, 8486, 10596, 12445, 13124, 15945],
    },
    Sound.SHORT_RATTLE_VARIANT_2_2: {
        SoundType.LOW: [83, 106, 262, 326, 475],
        SoundType.MIDDLE: [693],
        SoundType.HIGH: [1034, 1647, 2740, 3826, 5507, 6045, 6479, 7150, 8391, 10637, 12424, 13688],
    },
    Sound.SHORT_RATTLE_VARIANT_2_3: {
        SoundType.LOW: [282, 394],
        SoundType.MIDDLE: [604, 684],
        SoundType.HIGH: [963, 1175, 1754, 2248, 2948, 3941, 5536, 6092, 6586, 8368, 10404, 12401, 14158, 16031],
    },
    Sound.SHORT_REPEATED_RATTLES_1: {},
    Sound.SHORT_SCRATCH_1: {},
}

"""
Long scratch and pot hit long
middle scratch long rattle
Pot hit short Pot hit flam
"""
piece.reset()
(
    piece.synth_player.add_sound(Sound.LONG_RATTLE_1, f"{SOUND_PATH}/Long Rattle 1.flac", 0.167, 1.445)
    .add_sound(Sound.LONG_RATTLE_2, f"{SOUND_PATH}/Long Rattle 2.flac", 0.089, 1.345)
    .add_sound(Sound.LONG_SCRATCH_1, f"{SOUND_PATH}/Long Scratch 1.flac", 0.081, 0.850)
    .add_sound(Sound.LONG_SCRATCH_2, f"{SOUND_PATH}/Long Scratch 2.flac", 0.024, 0.926)
    .add_sound(Sound.MIDDLE_SCRATCH_1, f"{SOUND_PATH}/MIddle Scratch 1.flac", 0.047, 0.558)
    .add_sound(Sound.POT_HIT_LONG_1, f"{SOUND_PATH}/Pot Hit Long 1.flac", 0.006, 1.077)
    .add_sound(Sound.POT_HIT_LONG_2, f"{SOUND_PATH}/Pot Hit Long 2.flac", 0.019, 0.895)
    .add_sound(Sound.POT_HIT_SHORT_1, f"{SOUND_PATH}/Pot Hit Short 1.flac", 0.010, 0.180)
    .add_sound(Sound.POT_HIT_SHORT_2, f"{SOUND_PATH}/Pot Hit Short 2.flac", 0.105, 0.410)
    .add_sound(Sound.POT_HIT_SHORT_3, f"{SOUND_PATH}/Pot Hit Short 3.flac", 0.022, 0.165)
    .add_sound(Sound.POT_HIT_SHORT_4, f"{SOUND_PATH}/Pot Hit Short 4.flac", 0.036, 0.212)
    .add_sound(Sound.POT_HIT_SHORT_5, f"{SOUND_PATH}/Pot Hit Short 5.flac", 0.062, 0.218)
    .add_sound(Sound.POT_HIT_SHORT_6, f"{SOUND_PATH}/Pot Hit Short 6.flac", 0.057, 0.236)
    .add_sound(Sound.POT_HIT_SHORT_7, f"{SOUND_PATH}/Pot Hit Short 7.flac", 0.030, 0.213)
    .add_sound(Sound.POT_HIT_SHORT_8, f"{SOUND_PATH}/Pot Hit Short 8.flac", 0.088, 0.272)
    .add_sound(Sound.POT_HIT_SHORT_FLAM_1, f"{SOUND_PATH}/Pot Hit Short Flam 1.flac", 0.117, 0.383)
    .add_sound(Sound.POT_HIT_SHORT_FLAM_2, f"{SOUND_PATH}/Pot Hit Short Flam 2.flac", 0.150, 0.387)
    .add_sound(Sound.POT_HIT_SHORT_FLAM_3, f"{SOUND_PATH}/Pot Hit Short Flam 3.flac", 0.198, 0.353)
    .add_sound(Sound.SCRATCH_HIT_1, f"{SOUND_PATH}/Scratch Hit 1.flac", 0.015, 0.615)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_1_1, f"{SOUND_PATH}/Short Rattle Variant 1 1.flac", 0.079, 0.453)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_1_2, f"{SOUND_PATH}/Short Rattle Variant 1 2.flac", 0.222, 0.507)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_1_3, f"{SOUND_PATH}/Short Rattle Variant 1 3.flac", 0.206, 0.634)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_2_1, f"{SOUND_PATH}/Short Rattle Variant 2 1.flac", 0.073, 0.485)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_2_2, f"{SOUND_PATH}/Short Rattle Variant 2 2.flac", 0.193, 0.602)
    .add_sound(Sound.SHORT_RATTLE_VARIANT_2_3, f"{SOUND_PATH}/Short Rattle Variant 2 3.flac", 0.152, 0.493)
    .add_sound(Sound.SHORT_REPEATED_RATTLES_1, f"{SOUND_PATH}/Short Repeated Rattles 1.flac", 0.070, 1.377)
    .add_sound(Sound.SHORT_SCRATCH_1, f"{SOUND_PATH}/Short scratch 1.flac", 0.116, 0.368)
    .add_impulse_response("ir1", f"{IR_PATH}/stalbans_b_ortf-L.wav", f"{IR_PATH}/stalbans_b_ortf-R.wav")
    .add_impulse_response(
        "ir2", f"{IR_PATH}/falkland_tennis_court_ortf-L.wav", f"{IR_PATH}/falkland_tennis_court_ortf-R.wav"
    )
    .add_impulse_response("ir3", f"{IR_PATH}/5UnderpassValencia-L.wav", f"{IR_PATH}/5UnderpassValencia-R.wav")
    .add_impulse_response("ir4", f"{IR_PATH}/DrainageTunnel-L.wav", f"{IR_PATH}/DrainageTunnel-R.wav")
    .add_impulse_response("ir5", f"{IR_PATH}/HartwellTavern-L.wav", f"{IR_PATH}/HartwellTavern-R.wav")
    .add_impulse_response("ir6", f"{IR_PATH}/RacquetballCourt-L.wav", f"{IR_PATH}/RacquetballCourt-R.wav")
    .add_impulse_response("ir7", f"{IR_PATH}/stalbans_a_ortf-L.wav", f"{IR_PATH}/stalbans_a_ortf-R.wav")
    .add_impulse_response("ir8", f"{IR_PATH}/BatteryTolles-L.wav", f"{IR_PATH}/BatteryTolles-R.wav")
    .start()
)

high_pan_points = [(-0.99, -0.75), (0.75, 0.99)]
middle_pan_points = [(-0.66, -0.33), (0.33, 0.66)]
low_pan_points = [(-0.25, 0), (0, 0.25)]

def get_sound_duration(sound_name: str) -> float:
    return piece.synth_player.get_sound(sound_name).duration(1.0)

class PotHitShort:
    sound_group = SoundGroup.POT_HIT_SHORT

    low_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    middle_ring_chain = MarkovChain({
        True: {True: 0, False: 1},
        False: {True: 0.6, False: 0.4},
    }, False)

    high_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    @classmethod
    def handle_low(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 1))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.low_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot Hit Short Low", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_middle(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(0, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 3))))
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH], k=random_int_range(1, 3))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks), (high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.middle_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)

                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot Hit Short Middle", duration=get_sound_duration(sound), freq=sound_peak))
        return notes


    @classmethod
    def handle_high(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]        
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH][:7], k=random_int_range(3, 7))))
        start_time = current_time + random_range(-0.02, 0.02)
        notes = []
        for pan_points, sound_peaks in [(high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.high_ring_chain.next()
                
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)                    
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.mono_high_pass_filter(piece.control_instruments.static_control(min(sound_peaks)))
                    .pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot Hit Short High", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

class ShortRattleVariant1:
    sound_group = SoundGroup.SHORT_RATTLE_VARIANT_1

    low_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    middle_ring_chain = MarkovChain({
        True: {True: 0, False: 1},
        False: {True: 0.6, False: 0.4},
    }, False)

    high_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    @classmethod
    def handle_low(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 1))))     
        notes = []   
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.low_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 1 low", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_middle(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(0, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 3))))
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH], k=random_int_range(1, 3))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks), (high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.middle_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 1 middle", duration=get_sound_duration(sound), freq=sound_peak))
        return notes


    @classmethod
    def handle_high(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]        
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH][:7], k=random_int_range(3, 7))))
        notes = []
        for pan_points, sound_peaks in [(high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.high_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)                    
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.mono_high_pass_filter(piece.control_instruments.static_control(min(sound_peaks)))
                    .pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 1 high", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

class ShortRattleVariant2:
    sound_group = SoundGroup.SHORT_RATTLE_VARIANT_2

    low_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    middle_ring_chain = MarkovChain({
        True: {True: 0, False: 1},
        False: {True: 0.6, False: 0.4},
    }, False)

    high_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    @classmethod
    def handle_low(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 1))))     
        notes = []   
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.low_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 2 low", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_middle(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(0, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 3))))
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH], k=random_int_range(1, 3))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks), (high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.middle_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 2 middle", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_high(cls, current_time: float, effect: SynthNote) -> list[SequenceNote]:
        sound_group_sounds = sound_groups[cls.sound_group]
        sound = random.choice(sound_group_sounds)
        sound_types = sounds[sound]        
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH][:7], k=random_int_range(3, 7))))
        notes = []
        for pan_points, sound_peaks in [(high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.high_ring_chain.next()
                start_time = current_time + random_range(-0.02, 0.02)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak),
                        piece.control_instruments.static_control(rq),
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)                    
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))
                (
                    note.mono_high_pass_filter(piece.control_instruments.static_control(min(sound_peaks)))
                    .pan(piece.control_instruments.static_control(pan))
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Short rattle variant 2 high", duration=get_sound_duration(sound), freq=sound_peak))
        return notes
            

class LongScratch:
    sound_group = SoundGroup.LONG_SCRATCH
    sound_group_sounds = sound_groups[sound_group]

    high_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    middle_ring_chain = MarkovChain({
        True: {True: 0, False: 1},
        False: {True: 0.6, False: 0.4},
    }, False)

    low_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.4, False: 0.6},
    }, False)

    @classmethod
    def handle_high(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH][5:], k=random_int_range(1, 7))))
        notes = []
        for pan_points, sound_peaks in [(high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.high_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))                    
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))                                    
                (
                    note.mono_high_pass_filter(piece.control_instruments.static_control(min(sound_peaks)))
                    .pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Long Scratch high", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_middle(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(1, 2))))
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH], k=random_int_range(1, 5))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks), (high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.middle_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))                    
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))                    
                (
                    note.pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Long Scratch middle", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_low(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 1))))        
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor
                pan = pan_point(pan_points)
                should_ring = cls.low_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))                    
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))                    
                (
                    note.pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Long Scratch low", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

class PotHitLong:
    sound_group = SoundGroup.POT_HIT_LONG
    sound_group_sounds = sound_groups[sound_group]

    low_pot_hit_long_ring_chain = MarkovChain({
        True: {True: 0.5, False: 0.5},
        False: {True: 0.9, False: 0.1},
    }, False)

    middle_pot_hit_long_ring_chain = MarkovChain({
        True: {True: 0, False: 1},
        False: {True: 0.4, False: 0.6},
    }, False)


    high_pot_hit_long_ring_chain = MarkovChain({
        True: {True: 0.1, False: 0.9},
        False: {True: 0.6, False: 0.4},
    }, False)

    @classmethod
    def handle_high(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]         
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH][4:], k=random_int_range(3, 7))))
        notes = []
        for pan_points, sound_peaks in [(high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor                
                pan = pan_point(pan_points)
                should_ring = cls.high_pot_hit_long_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )                
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)                
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))                
                (
                    note.mono_high_pass_filter(piece.control_instruments.static_control(min(sound_peaks)))
                    .pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot hit long high", duration=get_sound_duration(sound), freq=sound_peak))
        return notes

    @classmethod
    def handle_middle(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(1, 2))))
        high_sound_peaks = list(set(random.choices(sound_types[SoundType.HIGH], k=random_int_range(1, 5))))
        notes = []
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks), (high_pan_points, high_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor                
                pan = pan_point(pan_points)
                should_ring = cls.middle_pot_hit_long_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))

                (
                    note.pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot hit long middle", duration=get_sound_duration(sound), freq=sound_peak))
            return notes

    @classmethod
    def handle_low(cls, current_time: float, effect: SynthNote):
        sound = random.choice(cls.sound_group_sounds)
        sound_types = sounds[sound]
        low_sound_peaks = list(set(random.choices(sound_types[SoundType.LOW], k=random_int_range(1, 2))))
        middle_sound_peaks = list(set(random.choices(sound_types[SoundType.MIDDLE], k=random_int_range(0, 1))))   
        notes = []     
        for pan_points, sound_peaks in [(low_pan_points, low_sound_peaks), (middle_pan_points, middle_sound_peaks)]:
            for sound_peak in sound_peaks:
                bw = random_range(500, 600)
                static_amp_factor = 2 * random_range(0.85, 1.15)
                rq = bw / sound_peak
                amp_factor = (1 / math.sqrt(rq)) * static_amp_factor                
                pan = pan_point(pan_points)
                should_ring = cls.low_pot_hit_long_ring_chain.next()
                start_time = current_time + random_range(-0.03, 0.03)
                note = (
                    piece.synth_player.note()
                    .sound_mono(str(sound), 1.0, piece.control_instruments.static_control(1.0))
                    .mono_band_pass_filter(
                        piece.control_instruments.static_control(sound_peak), piece.control_instruments.static_control(rq)
                    )
                    .mono_volume(piece.control_instruments.static_control(amp_factor))
                )
                if should_ring:
                    ring = random.choice(sound_peaks)
                    note = note.ring_modulate(piece.control_instruments.static_control(ring))

                (
                    note.pan(piece.control_instruments.static_control(pan))                    
                    .send_to_synth_note(effect, start_time=start_time)
                )
                notes.append(SequenceNote(start=start_time, track="Pot hit long low", duration=get_sound_duration(sound), freq=sound_peak))
        return notes


class ShortPotHitRattleGroup:
    low_sound_group_chain = MarkovChain({
        SoundGroup.POT_HIT_SHORT: {
            SoundGroup.POT_HIT_SHORT: 0,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.5,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.5,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_1: {
            SoundGroup.POT_HIT_SHORT: 0.5,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.2,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.3,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_2: {
            SoundGroup.POT_HIT_SHORT: 0.5,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.3,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.2,
        },
    }, SoundGroup.POT_HIT_SHORT)

    middle_sound_group_chain = MarkovChain({
        SoundGroup.POT_HIT_SHORT: {
            SoundGroup.POT_HIT_SHORT: 0.4,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.3,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.3,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_1: {
            SoundGroup.POT_HIT_SHORT: 0.6,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.1,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.3,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_2: {
            SoundGroup.POT_HIT_SHORT: 0.6,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.3,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.1,
        },
    }, SoundGroup.POT_HIT_SHORT)

    high_sound_group_chain = MarkovChain({
        SoundGroup.POT_HIT_SHORT: {
            SoundGroup.POT_HIT_SHORT: 0.2,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.4,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.4,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_1: {
            SoundGroup.POT_HIT_SHORT: 0.4,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.2,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.4,
        },
        SoundGroup.SHORT_RATTLE_VARIANT_2: {
            SoundGroup.POT_HIT_SHORT: 0.4,
            SoundGroup.SHORT_RATTLE_VARIANT_1: 0.4,
            SoundGroup.SHORT_RATTLE_VARIANT_2: 0.2,
        },
    }, SoundGroup.POT_HIT_SHORT)
    
    @classmethod
    def play_low_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        current_time = time
        number_of_notes = random_int_range(1, 2)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.low_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_SHORT:
                    notes.extend(PotHitShort.handle_low(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_1:
                    notes.extend(ShortRattleVariant1.handle_low(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_2:
                    notes.extend(ShortRattleVariant2.handle_low(current_time, effect))
            #current_time += random_range(0.5, 1.3)
            current_time += random_range(0.05, 0.2)
        return number_of_notes, notes

    @classmethod
    def play_middle_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        
        current_time = time
        number_of_notes = random_int_range(1, 3)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.middle_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_SHORT:
                    notes.extend(PotHitShort.handle_middle(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_1:
                    notes.extend(ShortRattleVariant1.handle_middle(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_2:
                    notes.extend(ShortRattleVariant2.handle_middle(current_time, effect))
            #current_time += random_range(0.1, 1)
            current_time += random_range(0.05, 0.2)
        return number_of_notes, notes

    @classmethod
    def play_high_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        current_time = time
        #number_of_notes = random_int_range(3, 7) 
        number_of_notes = random_int_range(1, 3)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.high_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_SHORT:
                    notes.extend(PotHitShort.handle_high(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_1:
                    notes.extend(ShortRattleVariant1.handle_high(current_time, effect))
                case SoundGroup.SHORT_RATTLE_VARIANT_2:
                    notes.extend(ShortRattleVariant2.handle_high(current_time, effect))
            #current_time += random_range(0.1, 0.6)
            current_time += random_range(0.05, 0.2)
        return number_of_notes, notes


class PotHitScratchGroup:
    low_sound_group_chain = MarkovChain({
            SoundGroup.POT_HIT_LONG: {SoundGroup.POT_HIT_LONG: 0, SoundGroup.LONG_SCRATCH: 1},
            SoundGroup.LONG_SCRATCH: {SoundGroup.POT_HIT_LONG: 0.4, SoundGroup.LONG_SCRATCH: 0.6},
        }, SoundGroup.POT_HIT_LONG)
    
    middle_sound_group_chain = MarkovChain({
            SoundGroup.POT_HIT_LONG: {SoundGroup.POT_HIT_LONG: 0, SoundGroup.LONG_SCRATCH: 1},
            SoundGroup.LONG_SCRATCH: {SoundGroup.POT_HIT_LONG: 0.4, SoundGroup.LONG_SCRATCH: 0.6},
        }, SoundGroup.POT_HIT_LONG)

    high_sound_group_chain = MarkovChain({
            SoundGroup.POT_HIT_LONG: {SoundGroup.POT_HIT_LONG: 0, SoundGroup.LONG_SCRATCH: 1},
            SoundGroup.LONG_SCRATCH: {SoundGroup.POT_HIT_LONG: 0.3, SoundGroup.LONG_SCRATCH: 0.7},
        }, SoundGroup.POT_HIT_LONG)
    
    @classmethod
    def play_low_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        current_time = time
        number_of_notes = random_int_range(1, 2)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.low_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_LONG:
                    notes.extend(PotHitLong.handle_low(current_time, effect))                    
                case SoundGroup.LONG_SCRATCH:
                    notes.extend(LongScratch.handle_low(current_time, effect))
            #current_time += random_range(0.8, 1.3)
            current_time += random_range(0.05, 0.3)
        return number_of_notes, notes

    @classmethod
    def play_middle_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        current_time = time
        number_of_notes = random_int_range(1, 3)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.middle_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_LONG:
                    notes.extend(PotHitLong.handle_middle(current_time, effect))
                case SoundGroup.LONG_SCRATCH:
                    notes.extend(LongScratch.handle_middle(current_time, effect))
            #current_time += random_range(0.5, 1)
            current_time += random_range(0.05, 0.2)
        return number_of_notes, notes

    @classmethod
    def play_high_group(cls, time: float, effect: SynthNote) -> tuple[int, list[SequenceNote]]:
        current_time = time
        #number_of_notes = random_int_range(2, 5)
        number_of_notes = random_int_range(1, 3)
        notes = []
        for _ in range(number_of_notes):
            sound_group = cls.high_sound_group_chain.next()
            match sound_group:
                case SoundGroup.POT_HIT_LONG:
                    notes.extend(PotHitLong.handle_high(current_time, effect))
                case SoundGroup.LONG_SCRATCH:
                    notes.extend(LongScratch.handle_high(current_time, effect))
            #current_time += random_range(0.3, 0.7)
            current_time += random_range(0.05, 0.2)
        return number_of_notes, notes

# 1 1 2 3 5 8 13 21 34 55 89 144
PART_LENGTH = 89

class PotHitScratchPart:

    @classmethod
    def make_low_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        reverb_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb("ir8", piece.control_instruments.static_control(1.0))
                .play(start_time, duration)
        )

        clean_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration)
        )
        return effect
    
    @classmethod
    def make_middle_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        reverb_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb("ir2", piece.control_instruments.static_control(1.0))
                .play(start_time, duration)
        )

        clean_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration)
        )
        return effect

    @classmethod
    def make_high_effect(cls, start_time: float, duration: float) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        reverb_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb("ir4", piece.control_instruments.static_control(1.0))
                .play(start_time, duration)
        )

        clean_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration)
        )
        return effect
    
    @classmethod
    def play_low_part(cls, start_time: float) -> list[SequenceNote]:                
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effect = cls.make_low_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_low_group(current_time, effect)
            notes.extend(group_notes)
            match number_of_notes:
                #case 2:
                #    current_time += (5 * random_range(0.85, 1.15))
                case 1:
                    current_time += (8 * random_range(0.85, 1.15))
                case 2:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes

    @classmethod
    def play_middle_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effect = cls.make_middle_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_middle_group(current_time, effect)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes
    
    @classmethod
    def play_high_part(cls, start_time: float) -> list[SequenceNote]:
        end_time = start_time + PART_LENGTH
        current_time = start_time + random_range(1, 3)
        effect = cls.make_high_effect(start_time, PART_LENGTH)
        notes = []
        while current_time < (end_time - 5):
            number_of_notes, group_notes = PotHitScratchGroup.play_high_group(current_time, effect)
            notes.extend(group_notes)
            match number_of_notes:
                case 1:
                    current_time += (5 * random_range(0.85, 1.15))
                case 2:
                    current_time += (8 * random_range(0.85, 1.15))
                case 3:
                    current_time += (13 * random_range(0.85, 1.15))
        return notes

class ShortPotHitRattlePart:

        @classmethod
        def make_low_effect(cls, start_time: float, duration: float) -> SynthNote:            
            effect = (
                piece.synth_player.note(NodeId.EFFECT)
                    .stereo_input())

            reverb_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.4))
                    .stereo_convolution_reverb("ir6", piece.control_instruments.static_control(1.0))
                    .play(start_time, duration)
            )

            clean_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.6))
                    .play(start_time, duration)
            )
            return effect
        
        @classmethod
        def make_middle_effect(cls, start_time: float, duration: float) -> SynthNote:            
            effect = (
                piece.synth_player.note(NodeId.EFFECT)
                    .stereo_input())

            reverb_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.4))
                    .stereo_convolution_reverb("ir1", piece.control_instruments.static_control(1.0))
                    .play(start_time, duration)
            )

            clean_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.6))
                    .play(start_time, duration)
            )
            return effect
        
        @classmethod
        def make_high_effect(cls, start_time: float, duration: float) -> SynthNote:            
            effect = (
                piece.synth_player.note(NodeId.EFFECT)
                    .stereo_input())

            reverb_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.4))
                    .stereo_convolution_reverb("ir7", piece.control_instruments.static_control(1.0))
                    .play(start_time, duration)
            )

            clean_effect = (
                piece.synth_player.note(NodeId.ROOM_EFFECT)
                    .input_from_note(effect)
                    .stereo_volume(piece.control_instruments.static_control(0.6))
                    .play(start_time, duration)
            )
            return effect
        
        @classmethod
        def play_low_part(cls, start_time: float) -> list[SequenceNote]:
            end_time = start_time + PART_LENGTH
            current_time = start_time + random_range(1, 3)
            effect = cls.make_low_effect(start_time, PART_LENGTH)
            notes = []
            while current_time < (end_time - 5):
                number_of_notes, group_notes = ShortPotHitRattleGroup.play_low_group(current_time, effect)
                notes.extend(group_notes)
                match number_of_notes:
                    case 1:
                        current_time += (5 * random_range(0.85, 1.15))
                    case 2:
                        current_time += (8 * random_range(0.85, 1.15))
                    #case 3:
                    #    current_time += (13 * random_range(0.85, 1.15))
            return notes

        @classmethod
        def play_middle_part(cls, start_time: float) -> list[SequenceNote]:
            end_time = start_time + PART_LENGTH
            current_time = start_time + random_range(1, 3)
            effect = cls.make_middle_effect(start_time, PART_LENGTH)
            notes = []
            while current_time < (end_time - 5):
                number_of_notes, group_notes = ShortPotHitRattleGroup.play_middle_group(current_time, effect)
                notes.extend(group_notes)
                match number_of_notes:
                    case 1:
                        current_time += (5 * random_range(0.85, 1.15))
                    case 2:
                        current_time += (8 * random_range(0.85, 1.15))
                    case 3:
                        current_time += (13 * random_range(0.85, 1.15))
            return notes

        @classmethod
        def play_high_part(cls, start_time: float) -> list[SequenceNote]:
            end_time = start_time + PART_LENGTH
            current_time = start_time + random_range(1, 3)
            effect = cls.make_high_effect(start_time, PART_LENGTH)
            notes = []
            while current_time < (end_time - 5):
                number_of_notes, group_notes = ShortPotHitRattleGroup.play_high_group(current_time, effect)
                notes.extend(group_notes)
                match number_of_notes:
                    #case 3:
                    case 1:
                        current_time += (5 * random_range(0.85, 1.15))
                    #case 4 | 5:
                    case 2:
                        current_time += (8 * random_range(0.85, 1.15))
                    #case 6 | 7:
                    case 3:
                        current_time += (13 * random_range(0.85, 1.15))
            return notes
        

class PartHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:  
        
        match patch_arguments.midi_note:
            case 48:
                ShortPotHitRattlePart.play_low_part(patch_arguments.start)
            case 49:
                ShortPotHitRattlePart.play_middle_part(patch_arguments.start)
            case 50:
                ShortPotHitRattlePart.play_high_part(patch_arguments.start)
            case 51:
                PotHitScratchPart.play_low_part(patch_arguments.start)                
            case 52:
                PotHitScratchPart.play_middle_part(patch_arguments.start)
            case 53:
                PotHitScratchPart.play_high_part(patch_arguments.start)


class GroupHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def make_effect(self, start_time: float, ir: str, duration: float = 5) -> SynthNote:
        effect = (
            piece.synth_player.note(NodeId.EFFECT)
                .stereo_input())

        reverb_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.4))
                .stereo_convolution_reverb(ir, piece.control_instruments.static_control(1.0))
                .play(start_time, duration)
        )

        clean_effect = (
            piece.synth_player.note(NodeId.ROOM_EFFECT)
                .input_from_note(effect)
                .stereo_volume(piece.control_instruments.static_control(0.6))
                .play(start_time, duration)
        )
        return effect
    
    def handle_note(self, patch_arguments: PatchArguments) -> None:
        start_delay = 0.5
        match patch_arguments.midi_note:
            case 48:
                effect = self.make_effect(patch_arguments.start, "ir6")
                ShortPotHitRattleGroup.play_low_group(patch_arguments.start + start_delay, effect)
            case 49:
                effect = self.make_effect(patch_arguments.start, "ir1")
                ShortPotHitRattleGroup.play_middle_group(patch_arguments.start + start_delay, effect)
            case 50:
                effect = self.make_effect(patch_arguments.start, "ir7")
                ShortPotHitRattleGroup.play_high_group(patch_arguments.start + start_delay, effect)
            case 51:
                effect = self.make_effect(patch_arguments.start, "ir8")
                PotHitScratchGroup.play_low_group(patch_arguments.start + start_delay, effect)
            case 52:
                effect = self.make_effect(patch_arguments.start, "ir2")
                PotHitScratchGroup.play_middle_group(patch_arguments.start + start_delay, effect)
            case 53:
                effect = self.make_effect(patch_arguments.start, "ir4")
                PotHitScratchGroup.play_high_group(patch_arguments.start + start_delay, effect)

my_handler = PartHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)

class PartType(Enum):
    LOW_SHORT_POT_HIT_RATTLE = 1
    MIDDLE_SHORT_POT_HIT_RATTLE = 2
    HIGH_SHORT_POT_HIT_RATTLE = 3
    LOW_POT_HIT_SCRATCH = 4
    MIDDLE_POT_HIT_SCRATCH = 5
    HIGH_POT_HIT_SCRATCH = 6

part_type_chain = MarkovChain({
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2
   },
   (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0.2,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   },
   (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): {
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0.2,
       (PartType.LOW_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.LOW_POT_HIT_SCRATCH): 0,
       (PartType.HIGH_SHORT_POT_HIT_RATTLE, PartType.HIGH_POT_HIT_SCRATCH): 0
   }
}, (PartType.MIDDLE_SHORT_POT_HIT_RATTLE, PartType.MIDDLE_POT_HIT_SCRATCH))

from typing import Callable

def step_handler(i: int, start: float) -> list[SequenceNote]:
    notes = []
    for part_type in part_type_chain.next():
        start_time = start + random_range(0.3, 0.7)
        match part_type:
            case PartType.LOW_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_low_part(start_time))
            case PartType.MIDDLE_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_middle_part(start_time))
            case PartType.HIGH_SHORT_POT_HIT_RATTLE:
                notes.extend(ShortPotHitRattlePart.play_high_part(start_time))
            case PartType.LOW_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_low_part(start_time))
            case PartType.MIDDLE_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_middle_part(start_time))
            case PartType.HIGH_POT_HIT_SCRATCH:
                notes.extend(PotHitScratchPart.play_high_part(start_time))
    return notes

NUMBER_OF_SEQUENCE_GROUPS = 3
#NUMBER_OF_SEQUENCES_IN_GROUP = 2
TOTAL_NUMBER_OF_SEQUENCES = 5
PAUSE_LENGTH = 13

last_sequenser: Sequencer = None
for i in range(TOTAL_NUMBER_OF_SEQUENCES):
    if (i % NUMBER_OF_SEQUENCE_GROUPS) == (NUMBER_OF_SEQUENCE_GROUPS - 1):        
        next_time = (PART_LENGTH + PAUSE_LENGTH) * random_range(0.85, 1.0)
    else:
        next_time = PART_LENGTH * random_range(0.85, 1.0)
    
    print(i, next_time)
    new_sequencer = (
        Sequencer(1)
        .add_step_handler(step_handler)
        .next_time_handler(lambda i: next_time)
    )

    if last_sequenser:
        new_sequencer.spawn_sequencer(0, last_sequenser)
        last_sequenser.start_time_handler(lambda start: start + next_time)
    last_sequenser = new_sequencer
        


notes = last_sequenser.generate(0)
#notes = []

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

min_freq = 0
max_freq = 0

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
        min_freq = min(min_freq, note.freq)
        max_freq = max(max_freq, note.freq)

    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 3
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)

from ipywidgets import Output

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()


@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)


with hold_canvas():

    canvas.clear()
    for track_index, track in enumerate(sorted(ui_piece.tracks, key=lambda tr: tr.track_name)):
        canvas.font = "14px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

import ipywidgets as widgets

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")

def stop_playback(b):
    piece.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)



0 88.70358844943384
1 75.78670929767313
2 96.35503289404504
3 85.35813504612273
4 81.56236869455682


{'total': 409.7024465479792,
 'total minutes': 6.828374109132986,
 'tracks': 15,
 'Short rattle variant 2 low': 228.50330813219853,
 'Short rattle variant 1 low': 241.58921568704616,
 'Pot Hit Short Low': 228.5662712120038,
 'Long Scratch low': 242.0110765445579,
 'Pot hit long low': 242.07160973826151,
 'Short rattle variant 2 high': 397.37402916512485,
 'Pot Hit Short High': 409.6111668351268,
 'Short rattle variant 1 high': 409.7024465479792,
 'Long Scratch high': 317.8463252101271,
 'Pot hit long high': 265.86856442402615,
 'Pot Hit Short Middle': 324.6039101214472,
 'Short rattle variant 1 middle': 312.553578686739,
 'Short rattle variant 2 middle': 324.7669093865862,
 'Long Scratch middle': 399.7652327660402,
 'Pot hit long middle': 399.8357054382219}

Canvas(height=1500, width=1429)

Button(description='Stop', style=ButtonStyle())

Output()

In [33]:
piece.stop()